In [1]:
import torch
import cv2
import time

# Load models
yolo_nano = torch.hub.load('ultralytics/yolov5', 'custom', path='/Users/ashish/Desktop/final_projects/suspicious activityy/code/nano_weights/best.pt')  # First-stage model
yolo_large = torch.hub.load('ultralytics/yolov5', 'custom', path='/Users/ashish/Desktop/final_projects/suspicious activityy/code/large_weights/best.pt')  # Second-stage model

# Two-stage detection pipeline
def two_stage_detection_pipeline(image_path):
    # Load image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # First-stage detection (YOLOv5 Nano)
    start_time_nano = time.time()
    results_nano = yolo_nano(image_rgb)
    end_time_nano = time.time()
    detections_nano = results_nano.xyxy[0]  # [x1, y1, x2, y2, confidence, class]

    verified_detections = []

    # Second-stage verification (YOLOv5 Large)
    start_time_large = time.time()
    for det in detections_nano:
        x1, y1, x2, y2, conf, cls = det.tolist()
        
        # Confidence threshold for Nano
        if conf > 0.25:  # Adjust threshold as needed
            # Crop region of interest (ROI) for second-stage model
            crop = image[int(y1):int(y2), int(x1):int(x2)]

            # Skip if crop is empty
            if crop.size == 0:
                continue

            # Convert crop to RGB format
            crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)

            # Run YOLOv5 Large on the cropped region without resizing
            results_large = yolo_large(crop_rgb, size=crop_rgb.shape[0])  # Use height of crop as size
            detections_large = results_large.xyxy[0]

            # Confirm detections
            for det_large in detections_large:
                x1_l, y1_l, x2_l, y2_l, conf_l, cls_l = det_large.tolist()
                
                if conf_l > 0.5:  # Adjust threshold for Large
                    # Map detection back to original coordinates
                    verified_detections.append([x1, y1, x2, y2, conf_l, cls_l])
    end_time_large = time.time()

    # Visualize or save results
    for x1, y1, x2, y2, conf, cls in verified_detections:
        label = yolo_large.names[int(cls)]
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(image, f"{label} {conf:.2f}", (int(x1), int(y1) - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
    # Show the final image
    cv2.imshow('Detections (Two-Stage)', image)
    cv2.waitKey(5000)
    cv2.destroyAllWindows()

    # Print inference times
    print(f"Two-Stage Pipeline:")
    print(f"  First-stage (Nano) inference time: {end_time_nano - start_time_nano:.2f} seconds")
    print(f"  Second-stage (Large) inference time: {end_time_large - start_time_large:.2f} seconds")

# Single-stage detection using YOLOv5 Large
def single_stage_detection_pipeline(image_path):
    # Load image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Large model detection without resizing to 640
    start_time = time.time()
    results_large = yolo_large(image_rgb, size=image_rgb.shape[0])  # Avoid resizing to 640
    end_time = time.time()

    detections_large = results_large.xyxy[0]  # [x1, y1, x2, y2, confidence, class]

    # Visualize or save results
    for det in detections_large:
        x1, y1, x2, y2, conf, cls = det.tolist()
        if conf > 0.5:  # Confidence threshold for Large
            label = yolo_large.names[int(cls)]
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(image, f"{label} {conf:.2f}", (int(x1), int(y1) - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the final image
    cv2.imshow('Detections (Single-Stage)', image)
    cv2.waitKey(5000)
    cv2.destroyAllWindows()

    # Print inference time
    print(f"Single-Stage Pipeline:")
    print(f"  YOLOv5 Large inference time: {end_time - start_time:.2f} seconds")

# Test pipeline
image_path = '/Users/ashish/Desktop/final_projects/suspicious activityy/dataset/OD-WeaponDetection/Weapons and similar handled objects/Sohas_weapon-Detection-YOLOv5/model_1_data/images/test/pistol_9019.jpg'
print("\n--- Running Two-Stage Pipeline ---")
two_stage_detection_pipeline(image_path)

print("\n--- Running Single-Stage Pipeline ---")
single_stage_detection_pipeline(image_path)


Using cache found in /Users/ashish/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-31 Python-3.11.5 torch-2.2.2 CPU

Fusing layers... 
YOLOv5n summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
Using cache found in /Users/ashish/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-31 Python-3.11.5 torch-2.2.2 CPU

Fusing layers... 
YOLOv5l summary: 267 layers, 46113663 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 



--- Running Two-Stage Pipeline ---


2025-03-29 11:51:30.504 python[20354:1010707] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-29 11:51:30.504 python[20354:1010707] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Two-Stage Pipeline:
  First-stage (Nano) inference time: 0.05 seconds
  Second-stage (Large) inference time: 0.09 seconds

--- Running Single-Stage Pipeline ---
Single-Stage Pipeline:
  YOLOv5 Large inference time: 0.68 seconds


In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import time

# Ensure the YOLO models are loaded
import torch
yolo_nano = torch.hub.load('ultralytics/yolov5', 'custom', path='/Users/ashish/Desktop/final_projects/suspicious activityy/code/nano_weights/best.pt')  # First-stage model
yolo_large = torch.hub.load('ultralytics/yolov5', 'custom', path='/Users/ashish/Desktop/final_projects/suspicious activityy/code/large_weights/best.pt')  # Second-stage model

# Directory containing images
image_directory = '/Users/ashish/Desktop/final_projects/suspicious activityy/dataset/OD-WeaponDetection/Weapons and similar handled objects/Sohas_weapon-Detection-YOLOv5/model_1_data/images/test'

# Two-stage detection pipeline
def two_stage_detection_pipeline(image_path):
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        return 0  # Handle invalid image

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # First-stage detection (YOLOv5 Nano)
    start_time_nano = time.time()
    results_nano = yolo_nano(image_rgb)
    end_time_nano = time.time()
    detections_nano = results_nano.xyxy[0]

    verified_detections = []

    # Second-stage verification (YOLOv5 Large)
    start_time_large = time.time()
    for det in detections_nano:
        x1, y1, x2, y2, conf, cls = det.tolist()

        # Confidence threshold for Nano
        if conf > 0.25:
            # Crop region of interest (ROI)
            crop = image[int(y1):int(y2), int(x1):int(x2)]

            if crop.size == 0:
                continue

            # Convert crop to RGB format
            crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)

            # Run YOLOv5 Large on the cropped region
            results_large = yolo_large(crop_rgb, size=crop_rgb.shape[0])
            detections_large = results_large.xyxy[0]

            for det_large in detections_large:
                x1_l, y1_l, x2_l, y2_l, conf_l, cls_l = det_large.tolist()
                if conf_l > 0.5:
                    verified_detections.append([x1, y1, x2, y2, conf_l, cls_l])
    end_time_large = time.time()

    # Return total inference time
    return (end_time_nano - start_time_nano) + (end_time_large - start_time_large)

# Single-stage detection pipeline
def single_stage_detection_pipeline(image_path):
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        return 0  # Handle invalid image

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Large model detection
    start_time = time.time()
    results_large = yolo_large(image_rgb, size=image_rgb.shape[0])
    end_time = time.time()

    # Return inference time
    return end_time - start_time

# Measure average inference times
def measure_inference_times():
    two_stage_times = []
    single_stage_times = []

    # Process all images in the directory
    for filename in os.listdir(image_directory):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(image_directory, filename)
            print(f"Processing {filename}...")

            # Measure two-stage pipeline time
            time_two_stage = two_stage_detection_pipeline(image_path)
            two_stage_times.append(time_two_stage)

            # Measure single-stage pipeline time
            time_single_stage = single_stage_detection_pipeline(image_path)
            single_stage_times.append(time_single_stage)

    # Calculate average times
    avg_two_stage = sum(two_stage_times) / len(two_stage_times) if two_stage_times else 0
    avg_single_stage = sum(single_stage_times) / len(single_stage_times) if single_stage_times else 0

    return avg_two_stage, avg_single_stage

# Plot results
def plot_inference_times(avg_two_stage, avg_single_stage):
    pipelines = ['Two-Stage Pipeline', 'Single-Stage Pipeline']
    avg_times = [avg_two_stage, avg_single_stage]

    plt.figure(figsize=(8, 6))
    plt.bar(pipelines, avg_times, color=['blue', 'orange'])
    plt.ylabel('Average Inference Time (seconds)', fontsize=12)
    plt.xlabel('Pipeline', fontsize=12)
    plt.title('Comparison of Inference Times', fontsize=14)
    plt.ylim(0, max(avg_times) + 0.1)  # Add space above the highest bar
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Add text on top of the bars
    for i, time in enumerate(avg_times):
        plt.text(i, time + 0.02, f"{time:.2f} s", ha='center', fontsize=10, color='black')

    # Display the plot
    plt.tight_layout()
    plt.show()

# Main function
if __name__ == "__main__":
    print("Measuring inference times...")
    avg_two_stage, avg_single_stage = measure_inference_times()
    print(f"Average inference time for Two-Stage Pipeline: {avg_two_stage:.2f} seconds")
    print(f"Average inference time for Single-Stage Pipeline: {avg_single_stage:.2f} seconds")

    print("Plotting results...")
    plot_inference_times(avg_two_stage, avg_single_stage)


Using cache found in /Users/ashish/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-31 Python-3.11.5 torch-2.2.2 CPU

Fusing layers... 
YOLOv5n summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
Using cache found in /Users/ashish/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-31 Python-3.11.5 torch-2.2.2 CPU

Fusing layers... 
YOLOv5l summary: 267 layers, 46113663 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 


Measuring inference times...
Processing img1261.jpg...
Processing billete_2221.jpg...
Processing knife_1043.jpg...
Processing monedero_1133.jpg...
Processing HBmframe00163.jpg...
Processing knife_16.jpg...
Processing billete_2194.jpg...
Processing smartphone_1041.jpg...
Processing img6452.jpg...
Processing KravMagaKnifeDefenseTechniques185.jpg...
Processing smartphone_1069.jpg...
Processing monedero_1523.jpg...
Processing billete_2355.jpg...
Processing ABmframe00262.jpg...
Processing pistol_5076.jpg...
Processing KravMagaKnifeDefenseTechniques153.jpg...
Processing smartphone_z3v2_28.jpg...
Processing knife_1136.jpg...
Processing monedero_1522.jpg...
Processing knife_1122.jpg...
Processing smartphone_1068.jpg...
Processing knife_576.jpg...
Processing img1102.jpg...
Processing ABsframe00208.jpg...
Processing knife_1308.jpg...
Processing knife_600.jpg...
Processing img859.jpg...
Processing knife_358.jpg...
Processing knife_416.jpg...
Processing billete_2208.jpg...
Processing knife_402.jpg

In [8]:
import matplotlib.pyplot as plt

# Replace these with the actual calculated values
avg_two_stage = 0.45  # Example value for the two-stage pipeline
avg_single_stage = 0.30  # Example value for the single-stage pipeline

# Function to plot the average inference times
def plot_inference_times(avg_two_stage, avg_single_stage):
    # Data for plotting
    pipelines = ['Two-Stage Pipeline', 'Single-Stage Pipeline']
    avg_times = [avg_two_stage, avg_single_stage]

    # Plot settings
    plt.figure(figsize=(8, 6))
    plt.bar(pipelines, avg_times, color=['blue', 'orange'])
    plt.ylabel('Average Inference Time (seconds)', fontsize=12)
    plt.xlabel('Pipeline', fontsize=12)
    plt.title('Comparison of Inference Times', fontsize=14)
    plt.ylim(0, max(avg_times) + 0.1)  # Add space above the highest bar
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Add text on top of the bars
    for i, time in enumerate(avg_times):
        plt.text(i, time + 0.02, f"{time:.2f} s", ha='center', fontsize=10, color='black')

    # Display the plot
    plt.tight_layout()
    plt.show()

# Call the function
plot_inference_times(avg_two_stage, avg_single_stage)


In [2]:
import torch
import cv2
import time

# Load models
yolo_nano = torch.hub.load('ultralytics/yolov5', 'custom', path='/Users/ashish/Desktop/final_projects/suspicious activityy/code/nano_weights/best.pt')
yolo_large = torch.hub.load('ultralytics/yolov5', 'custom', path='/Users/ashish/Desktop/final_projects/suspicious activityy/code/large_weights/best.pt')

def two_stage_detection_pipeline(frame):
    # First-stage detection
    results_nano = yolo_nano(frame)
    detections_nano = results_nano.xyxy[0]

    verified_detections = []

    # Second-stage verification
    for det in detections_nano:
        x1, y1, x2, y2, conf, cls = det.tolist()
        if conf > 0.4:  # Confidence threshold for Nano
            crop = frame[int(y1):int(y2), int(x1):int(x2)]
            if crop.shape[0] > 0 and crop.shape[1] > 0:
                results_large = yolo_large(crop)
                detections_large = results_large.xyxy[0]

                for det_large in detections_large:
                    x1_l, y1_l, x2_l, y2_l, conf_l, cls_l = det_large.tolist()
                    if conf_l > 0.5:  # Confidence threshold for Large
                        verified_detections.append([x1, y1, x2, y2, conf_l, cls_l])

    # Draw detections
    for x1, y1, x2, y2, conf, cls in verified_detections:
        label = yolo_large.names[int(cls)]
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {conf:.2f}", (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return frame

# Live feed
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    processed_frame = two_stage_detection_pipeline(frame_rgb)

    cv2.imshow('Live Detection', processed_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in /Users/ashish/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-31 Python-3.11.5 torch-2.2.2 CPU

Fusing layers... 
YOLOv5n summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
Using cache found in /Users/ashish/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-31 Python-3.11.5 torch-2.2.2 CPU

Fusing layers... 
YOLOv5l summary: 267 layers, 46113663 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 


In [2]:
import torch
import cv2
import time

# Load models
yolo_nano = torch.hub.load('ultralytics/yolov5', 'custom', path='/Users/ashish/Desktop/final_projects/suspicious activityy/code/nano_weights/best.pt')  # First-stage model
yolo_large = torch.hub.load('ultralytics/yolov5', 'custom', path='/Users/ashish/Desktop/final_projects/suspicious activityy/code/large_weights/best.pt')  # Second-stage model

# Two-stage detection pipeline
    verified_detections = []

    # First-stage detection (YOLOv5 Nano)
    start_time_nano = time.time()
    results_nano = yolo_nano(frame)
    detections_nano = results_nano.xyxy[0]  # [x1, y1, x2, y2, confidence, class]
    end_time_nano = time.time()

    # Second-stage verification (YOLOv5 Large)
    start_time_large = time.time()
    for det in detections_nano:
        x1, y1, x2, y2, conf, cls = det.tolist()

        # Confidence threshold for Nano
        if conf > 0.4:  # Adjust threshold as needed
            # Crop region of interest (ROI) for second-stage model
            crop = frame[int(y1):int(y2), int(x1):int(x2)]

            # Skip resizing in YOLOv5 Large by passing original crop size
            if crop.shape[0] > 0 and crop.shape[1] > 0:  # Ensure valid crop dimensions
                results_large = yolo_large(crop, size=max(crop.shape[:2]))
                detections_large = results_large.xyxy[0]

                # Confirm detections
                for det_large in detections_large:
                    x1_l, y1_l, x2_l, y2_l, conf_l, cls_l = det_large.tolist()

                    if conf_l > 0.5:  # Adjust threshold for Large
                        # Map detection back to original coordinates
                        verified_detections.append([x1, y1, x2, y2, conf_l, cls_l])
    end_time_large = time.time()

    # Visualize results on the frame
    for x1, y1, x2, y2, conf, cls in verified_detections:
        label = yolo_large.names[int(cls)]
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {conf:.2f}", (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Print inference times
    print(f"Two-Stage Pipeline:")
    print(f"  First-stage (Nano) inference time: {end_time_nano - start_time_nano:.2f} seconds")
    print(f"  Second-stage (Large) inference time: {end_time_large - start_time_large:.2f} seconds")

    return frame

# Live camera feed
def live_camera_pipeline():
    cap = cv2.VideoCapture(1)  # Open webcam (0 for default camera)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break

        # Convert frame to RGB for YOLO models
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Run the two-stage detection pipeline
        processed_frame = two_stage_detection_pipeline(frame_rgb)

        # Display the frame with detections
        cv2.imshow('Two-Stage Detection (Live)', processed_frame)

        # Press 'q' to exit the live feed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Start the live camera pipeline
live_camera_pipeline()


IndentationError: unexpected indent (4212752451.py, line 10)

In [ ]:
import torch
import cv2
import time

# Load models
yolo_nano = torch.hub.load('ultralytics/yolov5', 'custom', path='/Users/ashish/Desktop/final_projects/suspicious activityy/code/nano_weights/best.pt')  # First-stage model
yolo_large = torch.hub.load('ultralytics/yolov5', 'custom', path='/Users/ashish/Desktop/final_projects/suspicious activityy/code/large_weights/best.pt')  # Second-stage model

# Two-stage detection pipeline
def two_stage_detection_pipeline(image_path):
    # Load image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # First-stage detection (YOLOv5 Nano)
    start_time_nano = time.time()
    results_nano = yolo_nano(image_rgb)
    end_time_nano = time.time()
    detections_nano = results_nano.xyxy[0]  # [x1, y1, x2, y2, confidence, class]

    verified_detections = []

    # Second-stage verification (YOLOv5 Large)
    start_time_large = time.time()
    for det in detections_nano:
        x1, y1, x2, y2, conf, cls = det.tolist()
        
        # Confidence threshold for Nano
        if conf >= 0.25:  # Adjust threshold as needed
            # Crop region of interest (ROI) for second-stage model
            crop = image[int(y1):int(y2), int(x1):int(x2)]
            
            # Run YOLOv5 Large on the cropped region
            results_large = yolo_large(crop)
            detections_large = results_large.xyxy[0]

            # Confirm detections
            for det_large in detections_large:
                x1_l, y1_l, x2_l, y2_l, conf_l, cls_l = det_large.tolist()
                
                if conf_l > 0.5:  # Adjust threshold for Large
                    # Map detection back to original coordinates
                    verified_detections.append([x1, y1, x2, y2, conf_l, cls_l])
    end_time_large = time.time()

    # Visualize or save results
    for x1, y1, x2, y2, conf, cls in verified_detections:
        label = yolo_large.names[int(cls)]
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(image, f"{label} {conf:.2f}", (int(x1), int(y1) - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the final image
    cv2.imshow('Detections (Two-Stage)', image)
    cv2.waitKey(5000)
    cv2.destroyAllWindows()

    # Print inference times
    print(f"Two-Stage Pipeline:")
    print(f"  First-stage (Nano) inference time: {end_time_nano - start_time_nano:.2f} seconds")
    print(f"  Second-stage (Large) inference time: {end_time_large - start_time_large:.2f} seconds")

# Single-stage detection using YOLOv5 Large
def single_stage_detection_pipeline(image_path):
    # Load image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Large model detection
    start_time = time.time()
    results_large = yolo_large(image_rgb)
    end_time = time.time()

    detections_large = results_large.xyxy[0]  # [x1, y1, x2, y2, confidence, class]

    # Visualize or save results
    for det in detections_large:
        x1, y1, x2, y2, conf, cls = det.tolist()
        if conf > 0.5:  # Confidence threshold for Large
            label = yolo_large.names[int(cls)]
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(image, f"{label} {conf:.2f}", (int(x1), int(y1) - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the final image
    cv2.imshow('Detections (Single-Stage)', image)
    cv2.waitKey(5000)
    cv2.destroyAllWindows()

    # Print inference time
    print(f"Single-Stage Pipeline:")
    print(f"  YOLOv5 Large inference time: {end_time - start_time:.2f} seconds")

# Test pipeline
image_path = '/Users/ashish/Desktop/final_projects/suspicious activityy/dataset/OD-WeaponDetection/Weapons and similar handled objects/Sohas_weapon-Detection-YOLOv5/model_1_data/images/test/pistola_z3v5_09.jpg'
two_stage_detection_pipeline(image_path)

print("\n--- Running Single-Stage Pipeline ---")
single_stage_detection_pipeline(image_path)

